In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
from tqdm import tqdm_notebook as tqdm
import pandas as pd
from datetime import datetime

In [ ]:
# zipmap.net/California/Santa_Clara_County.html
# sunnyvale - 94085, 94086, 94087, 94089
# cupertino - 95014
# los altos - 94024, 94022
# saratoga - 95070
# redwood city - 94063, 94061
## added on 5/18
# santa clara - 95050, 95051, 95128
# west san jose - 95129
# campell - 95117


## Could be added later
# mountain view - 94040, 94041, 94043
# west san jose - 95129
# campell - 95130, 95008

In [ ]:
zipcodes = [94087, 94086, 94085, 94089, 95014, 94024, 94022, 95070, 94063,94061, 95050, 95051, 95128, 95129, 95117]

In [ ]:
#search for active proxy - https://free-proxy-list.net/
proxy_list = ["37.120.192.154:8080", "201.148.70.100:80", "118.69.50.154:80",
              "203.124.39.150:80", "104.154.143.77:3128", "143.0.92.162:80",
              "164.163.73.66:999", "138.197.157.32:8080", "42.3.51.114:80",
              "103.105.49.53:80", "52.149.71.249:80", "115.249.2.192:8080",
              "176.9.75.42:8080", "203.202.245.62:80", "139.99.135.214:80",
              "138.68.41.90:3128", "88.198.50.103:8080", "46.4.96.137;3128",
              "78.93.58.60:3128", "191.233.199.87:80", "138.68.161.14:3128"]

In [ ]:
df = pd.DataFrame(columns = ['zipcode','address', 'listing_type', 'price', 'num_bds', 'num_ba', 'sqft_num', 'details', 'url'])
proxy_index = 0
for zc in tqdm(zipcodes):
    list_card = []
    while len(list_card) == 0:
        proxy_use = proxy_list[proxy_index]
        print(proxy_use)
        proxy_index += 1
        url = f'https://www.zillow.com/homes/for_sale/{zc}/0_singlestory/days_sort'
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument(f'--proxy-server=http://{proxy_use}')
        driver = webdriver.Chrome(executable_path='./chromedriver', chrome_options=chrome_options)
        driver.get(url)
        time.sleep(30)
        html = driver.execute_script("return document.getElementsByTagName('html')[0].innerHTML")
        soup = BeautifulSoup(html, 'html.parser')
        driver.close()

        zipcode = zc
        list_card = soup.find_all(class_ = "list-card-info")
    proxy_index -= 1
    
    for i in list_card:
        url = i.find('a')['href']
        address = i.find(class_ = "list-card-addr").text
        listing_type = i.find(class_ = "list-card-type").text
        price = i.find(class_ = "list-card-price").text
        details = i.find(class_ = "list-card-details").text
    
        if 'bds' in details: 
            bds_loc = details.find('bds')
            ba_loc = details.find('ba')
            sqft_loc = details.find('sqft')

            num_bds = details[0:bds_loc].strip()
            num_ba = details[bds_loc+3:ba_loc].strip()
            sqft_num = details[ba_loc+2:sqft_loc].strip()
        elif 'bd' in details: 
            bds_loc = details.find('bd')
            ba_loc = details.find('ba')
            sqft_loc = details.find('sqft')

            num_bds = details[0:bds_loc].strip()
            num_ba = details[bds_loc+2:ba_loc].strip()
            sqft_num = details[ba_loc+2:sqft_loc].strip()
        else:
            num_bds = ''
            num_ba = ''
            sqft_num = ''

        row = [zipcode, address, listing_type, price, num_bds, num_ba, sqft_num, details, url]
        df.loc[len(df)] = row
    time.sleep(30)
    
    
df['extract_ts'] = str(datetime.now())

In [ ]:
df.shape

In [ ]:
df.groupby(['zipcode']).nunique()

In [ ]:
df.head()

In [ ]:
df.to_csv('data/zillow_2020-06-22.csv')